<a href="https://colab.research.google.com/github/asdfg12131/TextImprovement/blob/main/Recomdation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn

import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
import pandas as pd
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from matplotlib.cbook import file_requires_unicode
from PIL import Image
from sklearn.model_selection import train_test_split
import urllib.request
from torch.utils.data import DataLoader
import os.path
from os import path
import requests
from io import BytesIO

<ipython-input-1-ef16e35810c3>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Dataset from https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small

In [2]:
#%rm -rf /content/sample_data/data/train
#%rm -rf /content/sample_data/data/


In [ ]:
import time

Downloading data:

In [ ]:
!unzip /content/sample_data/dataimages.csv.zip
!unzip /content/sample_data/styles.csv.zip

Archive:  /content/sample_data/images.csv.zip
  inflating: images.csv              
Archive:  /content/sample_data/styles.csv.zip
  inflating: styles.csv              


In [3]:
images = pd.read_csv("/content/sample_data/images.csv")
images.head()

,filename,link
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [4]:
styles = pd.read_csv("/content/sample_data/styles.csv",on_bad_lines='skip')
styles.shape, images.shape

((44424, 10), (44446, 2))

In [5]:
counts = styles["articleType"].value_counts()
styles.shape

(44424, 10)

Delete all the things that are less than 100 in the data:

In [6]:
counts = styles["articleType"].value_counts()
new_style = styles["articleType"].map(counts)
styles1 = styles[new_style > 500]
styles1 = styles1[["id", "articleType"]]
styles1 = pd.DataFrame(styles1)
dataset = styles1
dataset.shape

(34223, 2)

Data processing:

In [7]:
images = images.rename(columns={"filename" : "id"})

In [8]:
images["id"] = images["id"].str.replace('.jpg', '', regex=True)

In [9]:
images = images.rename(columns={"filename" : "id"})

In [10]:
images["id"] = images["id"].astype("int")

In [11]:
images["id"].isna().sum()

0

In [12]:
images = images.rename(columns={"filename" : "id"})
images["id"] = images["id"].astype("int64")
images["id"].info
images.shape

(44446, 2)

Make styles.csv and images.csv the same size:

In [13]:
a = {}
for i in range(images.shape[0]):
  a[images["id"][i]] = images["link"][i]
a[15970]

'http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg'

In [14]:
dataset["id"].tail(1)

44423    51623
Name: id, dtype: int64

In [15]:
dataset["link"] = dataset["id"].map(a)
dataset.isna().sum()

id             0
articleType    0
link           0
dtype: int64

In [16]:
dataset = dataset.dropna()
dataset.isna().sum()


id             0
articleType    0
link           0
dtype: int64

Split into train and test:

In [17]:
train_set, val_set, y_train, y_val = train_test_split(dataset[["id", "link", "articleType"]], dataset[["id", "articleType"]] , test_size=0.20, random_state=42)

In [18]:
len(train_set), len(val_set)

(27378, 6845)

New folders for data:

In [19]:
!mkdir /content/sample_data/data
!mkdir /content/sample_data/data/train
!mkdir /content/sample_data/data/val

mkdir: cannot create directory ‘/content/sample_data/data’: File exists
mkdir: cannot create directory ‘/content/sample_data/data/train’: File exists
mkdir: cannot create directory ‘/content/sample_data/data/val’: File exists


In [20]:
#%rm -rf /content/sample_data/data/val


function for loading image from url and resizing into one shape for all images:


(Also collects them in special folders)

In [21]:
def load_image(link, phase, article, id):
    transform = torchvision.transforms.Resize((1800, 1800))

    response = requests.get(link)
    image = Image.open(BytesIO(response.content))
    image=image.convert('RGB')
    image = transform(image)
    file_path = "/content/sample_data/data/%s/%s/%d.jpg" % (phase, article, id)
    folder_path = "/content/sample_data/data/%s/%s" % (phase, article)

    if os.path.exists(folder_path):
      return image.save(file_path)
    else:
      os.mkdir(folder_path)
      return image.save(file_path)


In [22]:
def loading(dataset, n, phase):
  for i in range(n):
    load_image(dataset["link"].values[i], phase, dataset["articleType"].values[i], dataset["id"].values[i])


In [25]:
#%rm -rf /content/sample_data/data/train
#!mkdir /content/sample_data/data/val

collects them into one folder (train):

In [26]:
N = 50
for i in range(N):
  loading(train_set, i, "train")

collects them into one folder (validation set):

In [ ]:
N = 3
for i in range(10):
  loading(val_set, i, "val")

Transform images to tensors and normalize them:

In [27]:
# Преобразование обучающих данных для расширения обучающей выборки и её нормализация
# Для валидационной (тестовой) выборки только нормализация
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# папка с данными. Если запускаете в колабе, нужно скопировать данные к себе в директорию и примонтировать диск. Если запускаете локально -- просто скачайте данные
data_dir = '/content/sample_data/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
# специальный класс для загрузки данных в виде батчей
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

In [28]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7b00a7a2a5f0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7b00a7a2aad0>}

Model training:

In [29]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    #Ваш код здесь
    losses = {'train': [], "val": []}

    pbar = trange(num_epochs, desc="Epoch:")

    for epoch in pbar:

        # каждя эпоха имеет обучающую и тестовую стадии
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # установаить модель в режим обучения
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # итерируемся по батчам
            for data in tqdm(dataloaders[phase], leave=False):
                # получаем картинки и метки
                inputs, labels = data

                # оборачиваем в переменные
                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                else:
                    inputs, labels = inputs, labels

                # инициализируем градиенты параметров
                if phase=="train":
                    optimizer.zero_grad()

                # forward pass
                if phase == "eval":
                    with torch.no_grad():
                        outputs = model(inputs)
                else:
                    outputs = model(inputs)
                preds = torch.argmax(outputs, -1)
                loss = criterion(outputs, labels)

                # backward pass + оптимизируем только если это стадия обучения
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # статистика
                running_loss += loss.item()
                running_corrects += int(torch.sum(preds == labels.data))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            # Ваш код здесь
            losses[phase].append(epoch_loss)

            pbar.set_description('{} Loss: {:.4f} Acc: {:.4f}'.format(
                                    phase, epoch_loss, epoch_acc
                                ))

            # если достиглось лучшее качество, то запомним веса модели
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # загрузим лучшие веса модели
    model.load_state_dict(best_model_wts)
    return model, losses

In [30]:
print(dataset_sizes)

{'train': 49, 'val': 2}


In [31]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
loss_fn = nn.CrossEntropyLoss()

In [33]:
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

In [34]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
if torch.cuda.is_available():
    model.cuda()

In [ ]:
model, losses = train_model(model, loss_fn, optimizer_ft, exp_lr_scheduler, num_epochs=5)

Epoch::   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
model.eval()
convert_tensor = transforms.ToTensor()
transform = torchvision.transforms.Resize((1800, 1800))
link = val_set["link"].values[4000]
response = requests.get(link)
image = Image.open(BytesIO(response.content))
image=image.convert('RGB')
image = transform(image)
validate = convert_tensor(image)
with torch.no_grad():
  pred = model(validate)

In [ ]:
def evaluate(model):
    model.eval()

    runninig_correct = 0
    for data in dataloaders['val']:
        # получаем картинки и метки
        inputs, labels = data

        # переносим на gpu, если возможно
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()

        # forard pass
        output = model(inputs)
        _, predicted = torch.max(output, 1)

        runninig_correct += int(torch.sum(predicted == labels))

    return runninig_correct / dataset_sizes['val']